In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

# 🔹 환경변수 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 🔹 PDF → 벡터스토어 저장 함수
def load_and_store_pdf(file_path, persist_dir="vector_store"):
    loader = PyPDFLoader(file_path)
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)  # ⬅️ Token 문제 방지
    split_docs = splitter.split_documents(docs)

    embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small")
    vectordb = Chroma.from_documents(split_docs, embedding, persist_directory=persist_dir)
    vectordb.persist()
    print("✅ 벡터스토어 저장 완료")
    return vectordb

# 🔹 RAG QA 체인 반환 함수
def get_rag_chain(persist_dir="vector_store"):
    embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small")
    vectordb = Chroma(persist_directory=persist_dir, embedding_function=embedding)
    retriever = vectordb.as_retriever(search_kwargs={"k": 5})  # 검색 개수 제한

    llm = ChatOpenAI(model_name="gpt4", temperature=0.2, openai_api_key=OPENAI_API_KEY)
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=False  # ⬅️ token 초과 방지
    )
    return chain

# 🔹 질의응답 비교 함수
def compare_llm_vs_rag(query, rag_chain):
    # 기본 LLM 응답 (문서 검색 안 쓰고 순수 GPT만)
    llm = ChatOpenAI(model_name="gpt4", temperature=0.2, openai_api_key=OPENAI_API_KEY)
    llm_response = llm.predict(query)

    # RAG 응답 (문서 기반 검색)
    rag_response = rag_chain.run(query)

    return {
        "LLM 응답 (기본)": llm_response,
        "RAG 응답 (문서 기반)": rag_response
    }

# 🔹 테스트 예제 실행
if __name__ == "__main__":
    pdf_path = "KB 실버암 간편건강보험Plus.pdf"

    # 최초 실행 시: 벡터스토어 저장
    load_and_store_pdf(pdf_path)

    # RAG 체인 생성
    rag = get_rag_chain()

    # 예제 질의
    question = "갑상선암 진단일이 보함가입일로부터 며칠 후인가요?"
    result = compare_llm_vs_rag(question, rag)

    # 결과 출력
    for k, v in result.items():
        print(f"\n🔸 {k}:\n{v}")